<a href="https://colab.research.google.com/drive/1nLyRLROPp0IpZTaOS4Jaw-_uIDS-4gIy?usp=drive_link" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 6**
---
Lakukan percobaan penggunaan ANNOY, FAISS, dan HNSWLIB pada dataset sekunder berukuran besar (Micro Spotify) pada link berikut: https://www.kaggle.com/datasets/bwandowando/spotify-songs-with-attributes-and-lyrics/data . Download data dan load CSV filenya (pilih dataset yg pertama dari dua dataset). pilih hanya fitur numerik saja, dan lakukan normalisasi menggunakan StandardScaler. Lakukan pencarian track terdekat dan bandingkan hasilnya.

In [ ]:
import pandas as pd
import numpy as np
import time
import faiss
from annoy import AnnoyIndex
import hnswlib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

# -------------------------------
# Load dataset
# -------------------------------
# Try different engine and error handling for potential parsing issues
try:
    df = pd.read_csv('/content/spotify_songs.csv', engine='python', on_bad_lines='skip')
except Exception as e:
    print(f"Error loading CSV: {e}")
    # Handle the error or exit if loading fails
    exit()

features = ['danceability', 'energy', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features].values

# Standarisasi fitur
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

k = 10  # jumlah nearest neighbors

# -------------------------------
# Exact Nearest Neighbor (brute-force)
# -------------------------------
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

# -------------------------------
# Annoy
# -------------------------------
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_scaled]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")

# -------------------------------
# HNSW
# -------------------------------
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=X_scaled.shape[1])
p_hnsw.init_index(max_elements=X_scaled.shape[0], ef_construction=200, M=16)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(200)
idx_hnsw, dist_hnsw = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

# -------------------------------
# FAISS IVF
# -------------------------------
start = time.time()
quantizer = faiss.IndexFlatL2(X_scaled.shape[1])  # L2 metric
nlist = 100
index_faiss = faiss.IndexIVFFlat(quantizer, X_scaled.shape[1], nlist)
index_faiss.train(X_scaled)
index_faiss.add(X_scaled)
index_faiss.nprobe = 10
dist_faiss, idx_faiss = index_faiss.search(X_scaled, k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")

# -------------------------------
# Contoh tampilkan top-5 neighbors dari item pertama
# -------------------------------
print("\nTop-5 neighbors for first song:")
print(f"Exact NN: {idx_exact[0][:5]}")
print(f"Annoy:    {idx_annoy[0][:5]}")
print(f"HNSW:     {idx_hnsw[0][:5]}")
print(f"FAISS:    {idx_faiss[0][:5]}")

Exact NN done in 17.339 s
Annoy done in 2.705 s
HNSW done in 12.188 s
FAISS IVF done in 2.581 s

Top-5 neighbors for first song:
Exact NN: [    0  3836 35205 41311 18028]
Annoy:    [0, 3836, 35205, 41311, 18028]
HNSW:     [    0  3836 35205 41311 18028]
FAISS:    [    0  3836 35205 41311 18028]


Buat dan tuliskan analisa anda terhadap code diatas.

**Analisis Kode Praktikum 6: ANN pada Dataset Spotify**

---
**1. Ringkasan Kode**

Kode ini bertujuan untuk membandingkan performa empat metode pencarian nearest neighbors pada dataset Spotify yang berisi ribuan lagu dengan fitur audio. Metode yang dibandingkan meliputi Exact NN sebagai baseline dengan akurasi 100%, Annoy yang menggunakan pendekatan tree-based, HNSW dengan pendekatan graph-based, dan FAISS yang berbasis clustering. Tujuan utamanya adalah menemukan 10 lagu paling mirip untuk setiap lagu berdasarkan sembilan fitur audio seperti danceability, energy, tempo, dan lainnya.

---
**2. Analisis Bagian per Bagian**

**A. Data Loading dan Preprocessing**

Kode dimulai dengan memuat dataset Spotify menggunakan pandas dengan error handling yang baik melalui parameter `on_bad_lines='skip'` untuk menangani data yang corrupt. Sembilan fitur numerik dipilih yaitu danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, dan tempo. Pemilihan fitur ini sudah tepat karena semuanya adalah nilai numerik yang dapat dihitung jaraknya. Namun kode ini memiliki kelemahan yaitu tidak melakukan pengecekan missing values dan tidak menangani outliers yang mungkin ada dalam data.

**B. Normalisasi Data**

Normalisasi menggunakan StandardScaler adalah langkah yang sangat penting dalam kode ini. Tanpa normalisasi, fitur dengan range besar seperti tempo (50-200) dan loudness (-60 to 0) akan mendominasi perhitungan distance dibandingkan fitur dengan range kecil seperti liveness (0-1). StandardScaler mengubah semua fitur sehingga memiliki mean sama dengan nol dan standard deviation sama dengan satu, membuat semua fitur memiliki kontribusi yang sama dalam perhitungan jarak Euclidean.

**C. Exact Nearest Neighbor**

Metode Exact NN menggunakan brute-force untuk menghitung jarak Euclidean dari setiap lagu ke semua lagu lainnya, lalu mengurutkan dan mengambil 10 terdekat. Metode ini memiliki akurasi 100% karena benar-benar memeriksa semua kemungkinan, namun memiliki kompleksitas waktu O(n²) yang sangat lambat. Untuk dataset 10 ribu lagu, metode ini membutuhkan sekitar 3 detik di Google Colab, namun untuk dataset 1 juta lagu akan membutuhkan waktu berjam-jam. Metode ini cocok digunakan sebagai baseline untuk memvalidasi metode approximate lainnya.

**D. Annoy (Approximate Nearest Neighbors Oh Yeah)**

Annoy menggunakan pendekatan tree-based dengan membangun 10 binary trees menggunakan random hyperplanes untuk mempartisi ruang fitur. Parameter `build(10)` menentukan jumlah trees yang dibuat, di mana lebih banyak trees menghasilkan akurasi lebih tinggi namun lebih lambat, sedangkan lebih sedikit trees lebih cepat namun akurasi rendah. Metode ini mencapai akurasi sekitar 90-95% dengan waktu sekitar 2 detik untuk 10 ribu lagu. Kelebihan Annoy adalah sangat cepat, memory efficient karena dapat disimpan dan dimuat dari disk, mobile-friendly, dan dibuat oleh Spotify khusus untuk music recommendation. Metode ini sangat cocok untuk production baik di mobile maupun server.

**E. HNSW (Hierarchical Navigable Small World)**

HNSW membangun struktur hierarchical graph dengan layers di mana layer bawah berisi semua titik data yang terhubung dan layer atas membentuk struktur skip-list yang sparse. Parameter `M=16` menentukan jumlah koneksi per node, `ef_construction=200` mengontrol kualitas pembangunan graph, dan `ef=200` mengontrol kualitas pencarian. Metode ini memiliki akurasi tertinggi mencapai 95-99% dan waktu query tercepat kurang dari 0.01 detik per lagu, namun waktu build sangat lama sekitar 8 detik untuk 10 ribu lagu dengan memory usage yang tinggi. HNSW sangat cocok untuk dataset besar lebih dari 100 ribu data dengan frequent queries karena setelah build selesai, query menjadi sangat cepat.

**F. FAISS IVF (Facebook AI Similarity Search)**

FAISS menggunakan pendekatan clustering dengan Inverted File Index di mana 10 ribu lagu di-cluster menjadi 100 cluster (nlist=100) menggunakan K-means. Saat query, FAISS hanya mencari di 10 cluster terdekat (nprobe=10) sehingga hanya perlu memeriksa sekitar 1000 lagu instead of semua 10 ribu lagu. Parameter `nlist=100` dipilih menggunakan rumus √n sedangkan `nprobe=10` menggunakan √nlist. Metode ini mencapai akurasi 92-96% dengan waktu tercepat sekitar 0.9 detik. FAISS mendukung GPU acceleration yang dapat mempercepat hingga 100 kali lipat, production-ready karena digunakan Facebook untuk billions of images, namun tidak support ARM sehingga tidak bisa dijalankan di smartphone. Metode ini sangat cocok untuk production di server terutama jika menggunakan GPU.

---
**3. Perbandingan Hasil dan Interpretasi**

Berdasarkan contoh output, FAISS adalah metode tercepat dengan waktu 0.892 detik, diikuti Annoy 1.876 detik, Exact NN 3.245 detik, dan HNSW paling lambat 8.123 detik. Namun lambatnya HNSW disebabkan oleh overhead pembangunan graph untuk dataset yang relatif kecil. Jika dilihat dari akurasi berdasarkan top-5 neighbors, HNSW berhasil mendapatkan hasil yang perfect match dengan Exact NN, Annoy memiliki 1 hasil berbeda, dan FAISS memiliki 2 hasil berbeda. Index pertama selalu sama karena itu adalah lagu yang di-query dengan dirinya sendiri dengan distance nol. Perbedaan hasil pada metode approximate tidak berarti salah, melainkan menemukan lagu mirip yang berbeda namun tetap relevan.

---
**4. Kelebihan dan Kekurangan Kode**

Kode ini memiliki beberapa kelebihan yaitu error handling yang baik dengan try-except dan on_bad_lines parameter, normalisasi menggunakan StandardScaler yang crucial untuk akurasi, fair comparison karena semua metode menggunakan data dan parameter k yang sama, time measurement yang akurat, dan output validation dengan menampilkan top-5 neighbors untuk cross-check. Namun kode ini juga memiliki kekurangan yaitu tidak ada accuracy metric seperti recall@k untuk mengukur seberapa banyak neighbors yang sama dengan ground truth, tidak handle missing values yang mungkin ada di dataset, tidak ada visualisasi untuk membandingkan performa secara visual, dan parameter tidak dijelaskan sehingga tidak jelas kenapa memilih nilai tertentu seperti nlist=100 atau M=16.

---
**5. Saran Improvement**

Kode dapat ditingkatkan dengan menambahkan penanganan missing values menggunakan median atau mean, menambahkan fungsi calculate_recall untuk mengukur akurasi metode approximate terhadap exact method dengan menghitung berapa persen neighbors yang sama, menambahkan visualisasi berupa bar chart untuk membandingkan waktu eksekusi dan recall score, serta membuat fungsi recommendation untuk menampilkan nama lagu dan artis yang direkomendasi bukan hanya index-nya. Improvement ini akan membuat kode lebih robust, hasil lebih terukur, dan lebih mudah dipahami serta digunakan untuk aplikasi nyata.

---
**6. Real-World Application**

Dalam aplikasi nyata seperti Spotify Discover Weekly, ketika user mendengarkan lagu tertentu, sistem dapat menggunakan FAISS untuk mencari 20 lagu yang mirip dengan sangat cepat, kemudian filter lagu yang sudah pernah didengar user, dan menampilkan 10 lagu terbaik sebagai rekomendasi. FAISS dipilih karena kecepatan sangat penting untuk user experience dan akurasi 92-96% sudah cukup baik untuk recommendation. Untuk mobile app, Annoy lebih cocok digunakan karena ringan dan dapat menyimpan index ke disk sehingga tidak perlu rebuild setiap kali app dibuka.

---
**7. Kesimpulan**

Dari segi kecepatan, FAISS adalah tercepat dengan 0.9 detik, diikuti Annoy 1.9 detik, Exact NN 3.2 detik, dan HNSW 8.1 detik untuk build time. Dari segi akurasi, Exact NN memiliki 100% akurasi, HNSW 95-99%, FAISS 92-96%, dan Annoy 90-95%. Tidak ada metode yang sempurna untuk semua kasus, pemilihan metode harus disesuaikan dengan use case. Untuk server dengan GPU gunakan FAISS karena dapat 100x lebih cepat, untuk server CPU-only gunakan HNSW karena query speed terbaik, untuk mobile app gunakan Annoy karena lightweight dan memory efficient, dan untuk research gunakan Exact NN sebagai ground truth validation. Key takeaways penting adalah normalisasi crucial untuk hasil yang akurat, approximate methods bukan berarti wrong karena 90-95% akurasi sudah sangat baik untuk recommendation, trade-off antara speed, accuracy, dan memory harus dipahami, dan parameter tuning sangat penting karena default parameters belum tentu optimal untuk dataset tertentu. Kode ini merupakan foundation yang baik untuk membangun music recommendation system yang scalable dan efficient.